In [1]:
# necessary libraries for pre-processing
import utils
import pandas as pd
import numpy as np
import math
import os
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import preprocessing

In [2]:
pd.set_option('display.max_columns', 200)
# pd.set_option('display.width', 1000)

## Statistical Analysis

In [3]:
def get_col_max(df, col):
    '''Get the maximum value of a given column'''
    return df[col].max()

def get_col_min(df, col):
    '''Get the minimum value of a given column'''
    return df[col].min()

def get_col_count(df, col):
    '''Get the number of elements of a given column'''
    return df[col].count()

def get_col_avg(df, col):
    '''Get the average value of a given column'''
    return df[col].mean()

def get_col_std(df, col):
    '''Get the standard deviation value of a given column'''
    return df[col].std()

def get_col_cov(df, col):
    '''Get the covariance of the given column'''
    return np.cov(df[col])

def get_cov(series):
    '''Get the covariance of the given dataseries'''
    return np.cov(series)

def get_first(df):
    '''Get the first entry of a dataframe'''
    return df.iloc[0]

# Useful functions for preprocessing

In [4]:
def convert_date(df, column, date_format='%y%m%d'):
    '''Convert the given column containg dates in the given format
    to the standard date format and type'''
    copy_df = df.copy()
    copy_df[column] = pd.to_datetime(copy_df[column], format=date_format)

    return copy_df

In [5]:
def encode_column(df, column):
    '''Encode the given column of the given dataframe.'''
    copy_df = df.copy()

    le = preprocessing.LabelEncoder()
    le.fit(df[column].unique())
    copy_df[column] = le.transform(copy_df[column])
    
    return copy_df

In [6]:
def normalize_columns(df, columns):
    '''Normalize the given columns for range between [0, 1]'''
    for col in columns:
        col_min = get_col_min(df, col)
        col_max = get_col_max(df, col)
        
        df[col] = (df[col] - col_min)/\
                    (col_max - col_min)

In [7]:
def get_df_correlation(df, size=(11, 9)):
    '''Get the correlation between the dataframe features'''
    # Compute the correlation matrix
    corr = df.corr()

    # Generate a mask for the upper triangle
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True
    
    # Set up the matplotlib figure
    plt.subplots(figsize=size)

    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    # Draw the heatmap with the mask and correct aspect ratio
    ax = sns.heatmap(corr, mask=mask, cmap=cmap, center=0,
                     square=True, linewidths=.1, cbar_kws={"shrink": .5})
    
    y_lim = ax.get_ylim();
    ax.set_ylim(np.ceil(y_lim[0]), np.floor(y_lim[1]))

    plt.show()

In [8]:
def column_density_plot(df, col):
    '''Get a density plot for the given column in the given dataframe.
    Useful for outlier detection'''
    sns.kdeplot(df[col], shade=True)
    plt.show()

In [9]:
def scatterplot_two_cols(df, col1, col2):
    '''Get a scatterplot for the given two columns'''
    # Set up the matplotlib figure
    plt.subplots(figsize=(11, 9))
    
    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    sns.scatterplot(data=df, x=col1, y=col2,
                    hue='status',palette=cmap, sizes=(47,47))
    plt.show()

In [10]:
def get_features_histogram(df, figsize_val = (20, 25)):
    '''Get a histogram relating the features of the given dataframe'''
    fig = plt.figure(figsize = figsize_val)
    loan = df[df.status==1]
    do_not_loan = df[df.status==-1]
    j = 0

    for i in range(len(df.columns)):
        plt.subplot(math.ceil(len(df.columns) / 3), 3, j+1)
        j += 1
        sns.distplot(loan.iloc[:, i], color='g', label = 'loan',
                    kde = False if len(loan.iloc[:, i].unique()) is 1 else True)
        sns.distplot(do_not_loan.iloc[:, i], color='r', label = 'no loan',
                    kde = False if len(do_not_loan.iloc[:, i].unique()) is 1 else True)
        plt.legend(loc='best')

    fig.suptitle('Feature Analysis')
    fig.tight_layout()
    fig.subplots_adjust(top=0.95)
    plt.show()

## Imputation

In [11]:
def get_null_summary(dataset):
    '''Get a null summary display'''
    display(dataset.isnull().mean())

In [12]:
def clean_nulls(dataset, threshold=0.7):
    '''Clean nulls from the given table.
    If the nulls in a column are higher than the given threshold the entire column is deleted.
    If the nulls in a row are higher than the row, the row is also deleted.
    The threshold is a value between 0 and 1'''
    #Dropping columns with missing value rate higher than threshold
    dataset = dataset[dataset.columns[dataset.isnull().mean() < threshold]]

    #Dropping rows with missing value rate higher than threshold
    dataset = dataset.loc[dataset.isnull().mean(axis=1) < threshold]
    
    return dataset

In [13]:
def numerical_imputation(dataset, replacer=None):
    '''When null values exist, set them using the median of the colum,
    or a replacer, if one was given'''
    dataset = dataset.fillna(replacer if replacer is not None else dataset.median())

    return dataset

In [14]:
def categorical_imputation(dataset, column_name, replacer=None):
    '''Replace the inexistent values of the given column with the given replacer.
    If None replacer was ginve, use the column maximum value'''
    #Max fill function for categorical columns
    dataset[column_name].fillna(replacer if replacer is not None else \
                                dataset[column_name].value_counts()
                                                    .idxmax(),
                                inplace=True)
    
    return dataset

## Handling Outliers

In [15]:
def display_to_drop_std(dataset, column, mult_factor=3):
    '''Display the rows that will be dropped using the std approach'''
    upper_lim = dataset[column].mean() + dataset[column].std() * mult_factor
    lower_lim = dataset[column].mean() - dataset[column].std() * mult_factor

    display(dataset[(dataset[column] >= upper_lim) & (dataset[column] <= lower_lim)])

def drop_outliers_std(dataset, column, mult_factor=3):
    '''Drop the outlier rows with standard deviation'''
    upper_lim = dataset[column].mean() + dataset[column].std() * mult_factor
    lower_lim = dataset[column].mean() - dataset[column].std() * mult_factor

    return dataset[(dataset[column] < upper_lim) & (dataset[column] > lower_lim)]

In [16]:
def display_to_drop_percentile(dataset, column):
    '''Display the rows that will be dropped with Percentiles approach'''
    upper_lim = dataset[column].quantile(.95)
    lower_lim = dataset[column].quantile(.05)

    display(dataset[(dataset[column] >= upper_lim) & (dataset[column] <= lower_lim)])

def drop_outliers_percentile(dataset, column):
    '''Drop the outlier rows with Percentiles approach'''
    upper_lim = dataset[column].quantile(.95)
    lower_lim = dataset[column].quantile(.05)

    data = dataset[(dataset[column] < upper_lim) & (dataset[column] > lower_lim)]

## Binning

In [17]:
def numerical_binning(dataset):
    # TODO
    # https://towardsdatascience.com/feature-engineering-for-machine-learning-3a5e293a5114
    return False

In [18]:
def categorical_binning(dataset):
    # TODO
    return False

# Feature Engineering

### For a first simpler approach, we will only use the 'loan' table

In [19]:
# Loading the tables both used for train and test
account_df = utils.read_csv_to_df('competition_dataset/account.csv', delimiter=';')
disp_df = utils.read_csv_to_df('competition_dataset/disp.csv', delimiter=';')
client_df = utils.read_csv_to_df('competition_dataset/client.csv', delimiter=';')
demogra_df = utils.read_csv_to_df('competition_dataset/district.csv', delimiter=';')

# Loading the train tables
loan_df = utils.read_csv_to_df('competition_dataset/loan_train.csv', delimiter=';')
card_df = utils.read_csv_to_df('competition_dataset/card_train.csv', delimiter=';')
trans_df = utils.read_csv_to_df('competition_dataset/trans_train.csv', delimiter=';')

# Loading the test tables
loan_test_df = utils.read_csv_to_df('competition_dataset/loan_test.csv', delimiter=';')
card_test_df = pd.concat([utils.read_csv_to_df('competition_dataset/card_test.csv', delimiter=';'),
                          card_df])
trans_test_df = pd.concat([utils.read_csv_to_df('competition_dataset/trans_test.csv', delimiter=';'),
                            trans_df])

print(' ::: Tables Scheme :::')
print('\n\n\t LOAN TABLE')
display(loan_df.head())
print('\n\n\t ACCOUNT TABLE')
display(account_df.head())
print('\n\n\t DISPOSITION TABLE')
display(disp_df.head())
print('\n\n\t CREDIT CARD TABLE')
display(card_df.head())
print('\n\n\t CLIENT TABLE')
display(client_df.head())
print('\n\n\tTRANSACTIONS TABLE')
display(trans_df.head())
print('\n\n\tDEMOGRAPHIC TABLE')
display(demogra_df.head())

 ::: Tables Scheme :::


	 LOAN TABLE


/Users/edgarcarneiro/Documents/University/feup-ecac/project-competition/env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,loan_id,account_id,date,amount,duration,payments,status
0,5314,1787,930705,96396,12,8033,-1
1,5316,1801,930711,165960,36,4610,1
2,6863,9188,930728,127080,60,2118,1
3,5325,1843,930803,105804,36,2939,1
4,7240,11013,930906,274740,60,4579,1




	 ACCOUNT TABLE


,account_id,district_id,frequency,date
0,576,55,monthly issuance,930101
1,3818,74,monthly issuance,930101
2,704,55,monthly issuance,930101
3,2378,16,monthly issuance,930101
4,2632,24,monthly issuance,930102


...

,client_id,birth_number,district_id
0,1,706213,18
1,2,450204,1
2,3,406009,1
3,4,561201,5
4,5,605703,5




	TRANSACTIONS TABLE


,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,1548749,5270,930113,credit,credit in cash,800.0,800.0,NaN,NaN,NaN
1,1548750,5270,930114,credit,collection from another bank,44749.0,45549.0,NaN,IJ,80269753.0
2,3393738,11265,930114,credit,credit in cash,1000.0,1000.0,NaN,NaN,NaN
3,3122924,10364,930117,credit,credit in cash,1100.0,1100.0,NaN,NaN,NaN
4,1121963,3834,930119,credit,credit in cash,700.0,700.0,NaN,NaN,NaN




	DEMOGRAPHIC TABLE


,code,name,region,no. of inhabitants,no. of municipalities with inhabitants < 499,no. of municipalities with inhabitants 500-1999,no. of municipalities with inhabitants 2000-9999,no. of municipalities with inhabitants >10000,no. of cities,ratio of urban inhabitants,average salary,unemploymant rate '95,unemploymant rate '96,no. of enterpreneurs per 1000 inhabitants,no. of commited crimes '95,no. of commited crimes '96
0,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.29,0.43,167,85677,99107
1,2,Benesov,central Bohemia,88884,80,26,6,2,5,46.7,8507,1.67,1.85,132,2159,2674
2,3,Beroun,central Bohemia,75232,55,26,4,1,5,41.7,8980,1.95,2.21,111,2824,2813
3,4,Kladno,central Bohemia,149893,63,29,6,2,6,67.4,9753,4.64,5.05,109,5244,5892
4,5,Kolin,central Bohemia,95616,65,30,4,1,6,51.4,9307,3.85,4.43,118,2616,3040


# Individual pre processement of tables

In [34]:
def process_loans(loans_df, debug=False):
    '''Pre process the loan table'''
    if debug:
        print(' > Raw loan table representation')
        display(loans_df)
        print(' > Raw loan table correlations')
        get_df_correlation(loans_df)
        
    processed_df = convert_date(loans_df, 'date')
    
    # As there is a correlation between amount, duration & payments
    if debug:
        print(' > There is naturally a correlation between the amount, duration and payments columns,'+
              'since amount = columns * payments')
        scatterplot_two_cols(processed_df, 'amount', 'duration')
        scatterplot_two_cols(processed_df, 'amount', 'payments')
        print(' > Natural correlations, since the larger the amount the larger the monthly payment or the duration.')
        
        print(' > We can also use this correlation to inquiry if there is bank interest rate:')
        processed_df['payment_duration'] = processed_df['duration'] * processed_df['payments']
        scatterplot_two_cols(processed_df, 'amount', 'payment_duration')

        processed_df = processed_df.drop(['payment_duration'], axis=1)
        
    # No User has a history of loanning, at least in the train dataset
    # Since amount = payments * duration we can conclude that there is no interest rate to the bank
        
    return processed_df

In [21]:
def process_account(accounts_df, debug=False):
    '''Pre process the accounts table'''
    if debug:
        print(' > Raw accounts table representation')
        display(accounts_df)
        print(' > Raw accounts table correlations')
        get_df_correlation(accounts_df)
        
    processed_account = convert_date(account_df, 'date')
    
    # Encode categorical column
    df = encode_column(processed_account, 'frequency')
        
    if debug:
        print(' > Lets encode the frequency column:')
        display(df)
        print(' > And the features correlation:')
        get_df_correlation(df)

    return df

In [22]:
def process_dispostition(disp_df, debug=False):
    '''Pre process the dispositions table'''
    if debug:
        print(' > Raw disposition table representation')
        display(disp_df)
        print(' > Raw disposition table correlations')
        get_df_correlation(disp_df)
        
    processed_disp = disp_df.copy()
    
    # Renaming disp attributes
    processed_disp.loc[processed_disp["type"]=="OWNER","type"] = "O"
    processed_disp.loc[processed_disp["type"]=="DISPONENT","type"] = "U"
    
    # Transform numerical into categorical
    df = pd.get_dummies(processed_disp)
    
    if debug:
        print(' > Transformed the categorical type column into numerical respective columns')
        display(df)
        print(' > Since the 2 categories are depedent, we can remove one of them, for removing redundant data')
        get_df_correlation(df)
        
    # Cannot process further as needs merging with other columns
    return df

In [23]:
def process_card(card_df, debug=False):
    '''Pre process the credit card table'''
    if debug:
        print(' > Raw credit card table representation')
        display(card_df)
        print(' > Raw credit card table correlations')
        get_df_correlation(card_df)

    processed_card = convert_date(card_df, 'issued')

    # Renaming card attributes
    processed_card.loc[processed_card["type"]=="classic","type"] = "C"
    processed_card.loc[processed_card["type"]=="gold","type"] = "G"
    processed_card.loc[processed_card["type"]=="junior","type"] = "J"
    
    # Transform numerical into categorical & removing useless column
    df = pd.get_dummies(processed_card)
    df = df.drop(['card_id'], axis=1)
    
    if debug:
        print(' > Transformed the categorical type column into numerical respective columns')
        display(df)
        print(' > Since the 2 categories are depedent, we can remove one of them, for removing redundant data')
        get_df_correlation(df)
        
    # Cannot process further as needs merging with other columns
    return df

In [24]:
def process_client(client_df, debug=False):
    '''Pre process the client table'''
    if debug:
        print(' > Raw client table representation')
        display(client_df)
        print(' > Raw client table correlations')
        get_df_correlation(client_df)
        
    processed_df = client_df.copy()
        
    # Getting year, day, and month+50 if women
    processed_df['year'] = 1900 + (processed_df['birth_number'] // 10000)
    processed_df['month_gender'] = (processed_df['birth_number'] % 10000) // 100
    processed_df['day'] = processed_df['birth_number'] % 100

    # Extracting gender and month
    processed_df['gender'] = np.where(processed_df['month_gender']>=50, 1, 0)
    processed_df['month'] = np.where(processed_df['month_gender']>=50, processed_df['month_gender']-50, processed_df['month_gender'])

    # Composing data
    processed_df['birth_date'] = processed_df['year'] * 10000 +\
                                 processed_df['month'] * 100 +\
                                 processed_df['day']
    df = convert_date(processed_df, 'birth_date', '%Y%m%d')
    
    # Dropping useless columns
    df = df.drop(['birth_number', 'year', 'month_gender', 'month', 'day'], axis=1)
    
    if debug:
        print(' > After extracting the gender from the date we have:')
        display(df)
        print(' > Notice the gender representation:\n\t * 1 if female\n\t * 0 if male')
        get_df_correlation(df)
    
    return df

In [25]:
def process_transactions(transactions_df, debug=False):
    '''Pre process the transactions table'''
    if debug:
        print(' > Raw transactions table representation')
        display(transactions_df)
        print(' > Raw transactions table correlations')
        get_df_correlation(transactions_df)
        
    if debug:
        print(' > Null evaluation in transactions rows')
        get_null_summary(transactions_df)
        
    # Removing the null columns & processing data
    processed_df = clean_nulls(transactions_df)
    processed_df = convert_date(processed_df, 'date')
    
    # Filling null columns with too much nulls
    processed_df = categorical_imputation(processed_df, 'k_symbol', ' ')
    
    if debug:
        display(processed_df['operation'].unique())
    
    # Renaming 'withdrawal in cash' to 'withdrawal'
    processed_df.loc[processed_df["type"]=="withdrawal in cash","type"] = "withdrawal"

    # Renaming of operations
    processed_df.loc[processed_df["operation"]=="credit in cash", "operation"] = "CC"
    processed_df.loc[processed_df["operation"]=="collection from another bank", "operation"] = "CAB"
    processed_df.loc[processed_df["operation"]=="withdrawal in cash", "operation"] = "WC"
    processed_df.loc[processed_df["operation"]=="remittance to another bank", "operation"] = "RAB"
    processed_df.loc[processed_df["operation"]=="credit card withdrawal", "operation"] = "CCW"
    processed_df = categorical_imputation(processed_df, 'operation', 'IC') # Interest credited
    
    # Making withdrawals amount negatives
    processed_df.loc[processed_df["type"]=="withdrawal", "amount"] *=-1 
    
    if debug:
        print(' > Dataframe after renamings & deletion of empty columns')
        display(processed_df)
        display(processed_df['operation'].unique())
    
    # Aggregatting transaction balances
    agg_ballance = processed_df.sort_values(by=['account_id', 'date'],
                                            ascending=[True, False])\
                               .groupby(['account_id'])\
                               .agg({
                                    'balance': ['mean', 'max', 'min', 'std', get_first],
                                    'date': get_first,
                                    'amount': get_first
                               })\
                               .reset_index()
    agg_ballance.columns = ['account_id', 'balance_mean', 'balance_max', 'balance_min', 'balance_std',
                            'last_balance', 'last_trans_date', 'last_trans']

    # Agrregatting credits
    agg_credits = processed_df.sort_values(by=['account_id', 'date'],
                                            ascending=[True, False])\
                              .groupby(['account_id', 'type'])\
                              .agg({
                                  'amount': ['mean', 'count', 'max', 'min', 'std'],
                              })\
                              .reset_index()
    agg_credits.columns = ['account_id', 'type', 'credit_mean', 'credit_count',
                           'credit_max', 'credit_min', 'credit_std']
    agg_credits = agg_credits[agg_credits['type'] == 'credit']
    
    # Aggregatting withdrawals
    agg_withdrawals = processed_df.sort_values(by=['account_id', 'date'],
                                            ascending=[True, False])\
                                  .groupby(['account_id', 'type'])\
                                  .agg({
                                    'amount': ['mean', 'count', 'max', 'min', 'std'],
                                  })\
                                  .reset_index()
    agg_withdrawals.columns = ['account_id', 'type', 'withdrawal_mean', 'withdrawal_count',
                           'withdrawal_min', 'withdrawal_max', 'withdrawal_std']
    agg_withdrawals = agg_withdrawals[agg_withdrawals['type'] == 'withdrawal']
    
    # Aggregatting households
    households = processed_df.groupby(['account_id', 'k_symbol'])\
                             .agg({
                                 'amount': ['mean']
                             })\
                             .reset_index()
    households.columns = ['account_id', 'k_symbol', 'household']
    households = households[households['k_symbol'] == 'household']
    
    # Aggregatting households
    pensions = processed_df.groupby(['account_id', 'k_symbol'])\
                           .agg({
                               'amount': ['mean']
                           })\
                           .reset_index()
    pensions.columns = ['account_id', 'k_symbol', 'pension']
    pensions = pensions[pensions['k_symbol'] == 'old-age pension']

    # Aggregatting all the 5 tables into one - agg_ballance, agg_credits, agg_withdrawals, households & pensions
    df = agg_ballance.merge(agg_credits, on='account_id')\
                     .merge(agg_withdrawals, on='account_id', how='left')\
                     .merge(households, on='account_id', how='left')\
                     .merge(pensions, on='account_id', how='left')

    
    # Cleaning nulls and useless columns
    df = df.drop(['type_x', 'type_y', 'k_symbol_x', 'k_symbol_y'], axis=1)
    df = numerical_imputation(df, 0)

    
    # Cleaning nulls and performing aggreggation on table containing operations
    processed_df = pd.get_dummies(processed_df)
    operations_df = processed_df.groupby(['account_id'])\
                                .agg({
                                    'operation_CC': ['sum'],
                                    'operation_CAB': ['sum'],
                                    'operation_WC': ['sum'],
                                    'operation_RAB': ['sum'],
                                    'operation_CCW': ['sum'],
                                    'operation_IC': ['sum']
                                })\
                                .reset_index()
    operations_df.columns = ['account_id', 'operation_CC', 'operation_CAB',
                             'operation_WC', 'operation_RAB', 'operation_CCW', 'operation_IC']
    
    if debug:
        print(' > We can also compute the type operations for each account:')
        display(operations_df)
        
    # Join operations with the numeric data
    df = df.merge(operations_df, on='account_id', how='left')
    
    # Adding extra columns
    df['mean_trans_profit'] = df['credit_mean'] + df['withdrawal_mean']
    df['total_ops'] = df['operation_CC'] + df['operation_CAB'] + df['operation_WC']+\
                        df['operation_RAB'] + df['operation_CCW'] + df['operation_IC']
    df['credit_ratio'] = df['credit_count'] / df['total_ops']
    df['withdrawal_ratio'] = df['withdrawal_count'] / df['total_ops']
    df['balance_range'] = df['balance_max'] - df['balance_min']

    # OPs as ratios
    df['ratio_CC'] = df['operation_CC'] / df['total_ops']
    df['ratio_CAB'] = df['operation_CAB'] / df['total_ops']
    df['ratio_WC'] = df['operation_WC'] / df['total_ops']
    df['ratio_RAB'] = df['operation_RAB'] / df['total_ops']
    df['ratio_CCW'] = df['operation_CCW'] / df['total_ops']
    df['ratio_IC'] = df['operation_IC'] / df['total_ops']

    df = df.drop(['operation_CC', 'operation_CAB', 'operation_WC', 'operation_RAB',
                  'operation_CCW', 'operation_IC', 'credit_count', 'withdrawal_count'], axis=1)

    if debug:
        print(' > Table after processment of balance, credits, withdrawals, households, pensions and operations')
        display(df)
        get_null_summary(df)
        print(' > And the corresponding correlation matrix')
        get_df_correlation(df)
    
    return df

In [29]:
def process_demographic(demographic_df, debug=False):
    '''Pre process the demographic table'''
    if debug:
        print(' > Raw demographic table representation')
        display(demographic_df)
        print(' > Raw demographic table correlations')
        get_df_correlation(demographic_df)
        
    if debug:
        print(' > Null evaluation in demographic rows')
        get_null_summary(demographic_df)
    
    # Encode categorical columns
    df = encode_column(demographic_df, 'name ')
    df = encode_column(df, 'region')
    
    if debug:
        print('First, lets start by encoding the categorical columns')
        display(df)
        display(df.columns)

    # Replacing '?' by the average value
    median_crimes_95 = pd.to_numeric(df[df['no. of commited crimes \'95 '] != '?']\
                                     ['no. of commited crimes \'95 ']).median()
    df.loc[df['no. of commited crimes \'95 ']=="?", 'no. of commited crimes \'95 '] = median_crimes_95

    median_unemploymant_95 = (df[df['unemploymant rate \'95 '] != '?']\
                              ['unemploymant rate \'95 ']).astype(float).median()
    df.loc[df['unemploymant rate \'95 ']=="?", 'unemploymant rate \'95 '] = median_unemploymant_95
    
        
    # Transforming data to ints/ floats
    df['unemploymant rate \'95 '] = df['unemploymant rate \'95 '].astype(float)
    df['no. of commited crimes \'95 '] = pd.to_numeric(df['no. of commited crimes \'95 '])
    df['unemploymant rate \'96 '] = df['unemploymant rate \'96 '].astype(float)
    df['no. of commited crimes \'96 '] = pd.to_numeric(df['no. of commited crimes \'96 '])
    df['no. of enterpreneurs per 1000 inhabitants '] = pd.to_numeric(df['no. of enterpreneurs per 1000 inhabitants '])
    df['ratio of urban inhabitants '] = df['ratio of urban inhabitants '].astype(float)

    # Now lets create new features:
    df['ratio entrepeneurs'] = df['no. of enterpreneurs per 1000 inhabitants '] / 1000
    df['ratio of urban inhabitants '] = df['ratio of urban inhabitants '] / 100
    
    # Growths
    df['criminalty_growth'] = (df['no. of commited crimes \'96 '] - df['no. of commited crimes \'95 ']) /\
                              df['no. of inhabitants']
    df['unemploymant_growth'] = df['unemploymant rate \'96 '] - df['unemploymant rate \'95 ']
    
    df = df.drop(['no. of enterpreneurs per 1000 inhabitants ', 
                 'unemploymant rate \'96 ', 'no. of commited crimes \'96 ',
                  'no. of commited crimes \'95 '], axis=1)
    
    if debug:
        print('The final demographic plot, with additional features and having converted some features to ratios:')
        display(df)
        print('And the corresponding correlation matrix:')
        get_df_correlation(df)

    return df.rename(columns={'code ': 'district_id'}).drop(['name '], axis=1)

# Composite pre processment of tables

In [47]:
def compose_dataset(loan_df, account_df, disp_df, card_df, client_df, trans_df, demogra_df,
                    debug=False, pre_debug=False):
    '''Join the different tables and apply feature engineering'''

    # Pre processment of all the necessary tables
    processed_loan = process_loans(loan_df, pre_debug)
    processed_account = process_account(account_df, pre_debug)
    processed_trans = process_transactions(trans_df, pre_debug)
    processed_disp = process_dispostition(disp_df, pre_debug)
    processed_client = process_client(client_df, pre_debug)
    processed_card = process_card(card_df, pre_debug)
    processed_demogra = process_demographic(demogra_df, pre_debug)

    # Joining the different tables
    main_df = processed_loan.merge(processed_account.rename(columns={'date': 'account_creation_date'}),
                                   on='account_id', how='left')\
                            .merge(processed_trans,
                                   on='account_id', how='left')

    # Adding banck demographic info has negative impact on model performance,
    # since we are feeding inrelevant features
    main_df = main_df.drop(['district_id'], axis=1)

    if debug:
        print(' > Joint table of loans, account, transaction:')
        display(main_df)
        
    ################################################

    df_disp_client_card = processed_disp.merge(processed_client,
                                              on='client_id', how='left')\
                                        .merge(processed_card,
                                              on='disp_id', how='left')
    if debug:
        print(' > Joint table of disposition, client & card:')
        display(df_disp_client_card)
        get_null_summary(df_disp_client_card)
    
    # Since there are so many, we disconsider the client columns
    df_disp_client = df_disp_client_card.drop(['issued', 'type_C', 'type_G', 'type_J'], axis=1)
    
    # Now we aggreggate the remaining data
    # TODO: not yet assure how to handle agg on birthdate and Gender -> using account and district id to get owner
    df_disp_client = df_disp_client.sort_values(by=['account_id', 'type_O'], ascending=[True, False])\
                                   .groupby(['account_id'])\
                                   .agg({
                                       'type_O': ['count'],
                                       'type_U': ['count'],
                                       'gender': get_first,
                                       'birth_date': get_first,
                                       'district_id': get_first
                                   })\
                                   .reset_index()
    df_disp_client.columns= ['account_id', 'owner_count', 'disponent_count',
                             'owner_gender', 'owner_birthdate', 'district_id']
    
    if debug:
        print(' > Table after aggreggating data by account_id:')
        display(df_disp_client)
        
    # Now we join the district information regarding the owner
    df_secondary = df_disp_client.merge(processed_demogra, on='district_id')
    
    if debug:
        print(' > Table after joining with the demographics table:')
        display(df_secondary)
    
    ################################################
    
    # Joining the previously built two major tables
    df = main_df.merge(df_secondary, on='account_id')

    # Creating new columns using previous ones
    df['account_age_on_loan'] = (df['date'] - df['account_creation_date']).dt.days
    df['days_since_last_transaction'] = (df['date'] - df['last_trans_date']).dt.days
    df['owner_age_on_loan'] = (df['date'] - df['owner_birthdate']).dt.days / 365
    df["max_value_in_account_to_loan_ratio"] = df["balance_max"] / df["amount"]
    df["trans_per_day"] = df['total_ops'] / df["account_age_on_loan"]
    df["date"] = pd.to_numeric(df["date"])
    # Average salary, pension and household are anual- household value is negative
    df['expected_month_income'] = df['average salary '] - df['household'] / 12
    df.loc[df["pension"] > 0, "expected_month_income"] = df["pension"] - df['household'] / 12
    df['income_to_payments_ratio'] = df['expected_month_income'] / df['payments']

    # Showing some stats regarding the new features
    if pre_debug:
        print(' > Age of users when requesting a loan, in years')
        column_density_plot(df, 'owner_age_on_loan')
    
    # Dropping useless columns & normalizing others
    df = df.drop(['district_id', 'account_id', 'owner_birthdate', 'duration', 'average salary ',
                 'pension', 'household', 'expected_month_income'], axis=1)
    normalize_columns(df, ['date', 'account_creation_date'])

    if debug:
        print(' > Obtained dataframe after joining the previous tables and doing some feature engineering over them')
        display(df)
        print(' > And the corresponding correlation matrix:')
        get_df_correlation(df, (15, 15))

    # Writing loan_id for later association on predictions
    df = df.drop(['account_creation_date', 'last_trans_date', 'total_ops', 'disponent_count'], axis=1)
    
    # Setting status as last column
    df =  df[[col for col in df if col not in ['status']] + ['status']]
    
    if debug: 
        print(' >>> Final table that will serve as input to the model:')
        display(df)
        print(' > And the respective correlations')
        get_df_correlation(df, (15, 15))
        print(' > And the respective histograms for the given features')
        get_features_histogram(df, (20, 60))

    return df

dataset = compose_dataset(loan_df, account_df, disp_df, card_df, client_df,
                          trans_df, demogra_df, debug=True, pre_debug=False)

 > Joint table of loans, account, transaction:


,loan_id,account_id,date,amount,duration,payments,status,frequency,account_creation_date,balance_mean,balance_max,balance_min,balance_std,last_balance,last_trans_date,last_trans,credit_mean,credit_max,credit_min,credit_std,withdrawal_mean,withdrawal_min,withdrawal_max,withdrawal_std,household,pension,mean_trans_profit,total_ops,credit_ratio,withdrawal_ratio,balance_range,ratio_CC,ratio_CAB,ratio_WC,ratio_RAB,ratio_CCW,ratio_IC
0,5314,1787,1993-07-05,96396,12,8033,-1,2,1993-03-22,12250.000000,20100.0,1100.0,8330.866301,20100.0,1993-06-20,3300.0,5025.000000,9900.0,1100.0,3774.806838,0.000000,0.0,0.0,0.000000,0.0,0.0,5025.000000,4,1.000000,0.000000,19000.0,1.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,5316,1801,1993-07-11,165960,36,4610,1,1,1993-02-13,52083.859459,120512.8,700.0,29122.059454,52208.9,1993-07-09,-3419.0,13523.158824,36574.0,2.9,13998.092225,-8884.240000,-14.6,-54300.0,14074.800608,-4167.0,0.0,4638.918824,37,0.459459,0.540541,119812.8,0.324324,0.000000,0.324324,0.216216,0.0,0.135135
2,6863,9188,1993-07-28,127080,60,2118,1,1,1993-02-08,30060.954167,49590.4,800.0,11520.184451,20272.8,1993-07-21,-12000.0,5009.733333,19065.0,48.6,6801.234716,-6097.000000,-66.0,-14800.0,5390.565972,-7936.0,0.0,-1087.266667,24,0.625000,0.375000,48790.4,0.083333,0.208333,0.250000,0.125000,0.0,0.333333
3,5325,1843,1993-08-03,105804,36,2939,1,1,1993-01-30,41297.480000,65898.5,1000.0,14151.260443,34307.3,1993-07-31,178.1,9254.600000,26448.0,132.8,9818.591218,-7168.100000,-14.6,-15600.0,4919.096434,-6994.0,0.0,2086.500000,25,0.520000,0.480000,64898.5,0.080000,0.240000,0.400000,0.080000,0.0,0.200000
4,7240,11013,1993-09-06,274740,60,4579,1,2,1993-02-14,57188.211111,122893.1,600.0,25256.665817,41112.9,1993-08-31,-30.0,21255.930769,63366.0,77.1,24264.229780,-16801.000000,-30.0,-36700.0,13981.924936,-756.0,0.0,4454.930769,27,0.481481,0.518519,122293.1,0.037037,0.222222,0.407407,0.111111,0.0,0.222222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,6818,9030,1996-12-12,155616,48,3242,1,1,1995-01-21,44197.509884,75294.1,200.0,11044.494392,60694.1,1996-12-11,-14600.0,6448.582857,26388.0,23.7,9143.297044,-3830.458824,-14.6,-36960.0,5002.156631,-2587.0,0.0,2618.124034,172,0.406977,0.593023,75094.1,0.151163,0.000000,0.383721,0.209302,0.0,0.255814
324,5625,3189,1996-12-15,222180,60,3703,-1,1,1995-11-29,55230.444068,130659.1,800.0,26510.559286,59578.8,1996-12-10,-6900.0,13417.557143,44352.0,103.2,15547.006686,-10197.180645,-14.6,-52600.0,12259.070951,0.0,0.0,3220.376498,59,0.474576,0.525424,129859.1,0.271186,0.000000,0.525424,0.000000,0.0,0.203390
325,6805,8972,1996-12-21,45024,48,938,1,1,1996-05-21,41994.907692,63659.3,800.0,13151.510254,38384.3,1996-12-17,-17800.0,8544.930000,31636.5,41.0,12066.488804,-6974.431579,-14.6,-22100.0,7718.589020,-4719.0,0.0,1570.498421,39,0.512821,0.487179,62859.3,0.205128,0.000000,0.333333,0.153846,0.0,0.307692
326,7233,10963,1996-12-25,115812,36,3217,1,1,1995-05-20,56646.516129,119527.2,1100.0,21971.162852,41878.1,1996-12-18,-3100.0,16554.986275,49887.0,51.1,17170.849461,-10992.139726,-14.6,-50800.0,12328.776286,0.0,0.0,5562.846548,124,0.411290,0.588710,118427.2,0.258065,0.000000,0.588710,0.000000,0.0,0.153226


 > Joint table of disposition, client & card:


,disp_id,client_id,account_id,type_O,type_U,district_id,gender,birth_date,issued,type_C,type_G,type_J
0,1,1,1,1,0,18,1,1970-12-13,NaT,NaN,NaN,NaN
1,2,2,2,1,0,1,0,1945-02-04,NaT,NaN,NaN,NaN
2,3,3,2,0,1,1,1,1940-10-09,NaT,NaN,NaN,NaN
3,4,4,3,1,0,5,0,1956-12-01,NaT,NaN,NaN,NaN
4,5,5,3,0,1,5,1,1960-07-03,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5364,13647,13955,11349,1,0,1,1,1945-10-30,NaT,NaN,NaN,NaN
5365,13648,13956,11349,0,1,1,0,1943-04-06,NaT,NaN,NaN,NaN
5366,13660,13968,11359,1,0,61,0,1968-04-13,1995-06-13,1.0,0.0,0.0
5367,13663,13971,11362,1,0,67,1,1962-10-19,NaT,NaN,NaN,NaN


disp_id        0.000000
client_id      0.000000
account_id     0.000000
type_O         0.000000
type_U         0.000000
district_id    0.000000
gender         0.000000
birth_date     0.000000
issued         0.967033
type_C         0.967033
type_G         0.967033
type_J         0.967033
dtype: float64

 > Table after aggreggating data by account_id:


,account_id,owner_count,disponent_count,owner_gender,owner_birthdate,district_id
0,1,1,1,1,1970-12-13,18
1,2,2,2,0,1945-02-04,1
2,3,2,2,0,1956-12-01,5
3,4,1,1,0,1919-09-22,12
4,5,1,1,0,1929-01-25,15
...,...,...,...,...,...,...
4495,11333,1,1,0,1942-01-01,8
4496,11349,2,2,1,1945-10-30,1
4497,11359,1,1,0,1968-04-13,61
4498,11362,1,1,1,1962-10-19,67


 > Table after joining with the demographics table:


,account_id,owner_count,disponent_count,owner_gender,owner_birthdate,district_id,region,no. of inhabitants,no. of municipalities with inhabitants < 499,no. of municipalities with inhabitants 500-1999,no. of municipalities with inhabitants 2000-9999,no. of municipalities with inhabitants >10000,no. of cities,ratio of urban inhabitants,average salary,unemploymant rate '95,ratio entrepeneurs,criminalty_growth,unemploymant_growth
0,1,1,1,1,1970-12-13,18,5,70699,60,13,2,1,4,0.653,8968,2.83,0.131,0.002405,0.52
1,343,1,1,0,1978-03-13,18,5,70699,60,13,2,1,4,0.653,8968,2.83,0.131,0.002405,0.52
2,413,1,1,1,1935-07-08,18,5,70699,60,13,2,1,4,0.653,8968,2.83,0.131,0.002405,0.52
3,431,1,1,0,1980-04-13,18,5,70699,60,13,2,1,4,0.653,8968,2.83,0.131,0.002405,0.52
4,568,1,1,0,1979-10-21,18,5,70699,60,13,2,1,4,0.653,8968,2.83,0.131,0.002405,0.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,7990,1,1,1,1967-03-17,9,1,81344,61,23,4,2,6,0.553,8899,3.39,0.149,-0.006147,0.58
4496,8039,1,1,0,1972-06-23,9,1,81344,61,23,4,2,6,0.553,8899,3.39,0.149,-0.006147,0.58
4497,8153,1,1,0,1974-04-23,9,1,81344,61,23,4,2,6,0.553,8899,3.39,0.149,-0.006147,0.58
4498,9153,2,2,0,1938-09-25,9,1,81344,61,23,4,2,6,0.553,8899,3.39,0.149,-0.006147,0.58


KeyError: "['average_salary ' 'expected_month_income'] not found in axis"

In [44]:
# Outputting the resultant table to a final csv
utils.write_df_to_csv(dataset, 'dataset', 'preprocessed_data.csv')

test_dataset = compose_dataset(loan_test_df, account_df, disp_df,
                               card_test_df, client_df, trans_test_df, demogra_df)
utils.write_df_to_csv(test_dataset, 'dataset', 'test_dataset.csv')